In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("shift_students").getOrCreate()

In [2]:
# Create input data
data = [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie'),
    (4, 'David'),
    (5, 'Eve')
]

# Define schema
schema = ["id", "student"]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Register DataFrame as a temporary view
df.createOrReplaceTempView("students")

df.show()

+---+-------+
| id|student|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  4|  David|
|  5|    Eve|
+---+-------+



find previous and next student using lad and lead functions

In [4]:
from pyspark.sql import Window
from pyspark.sql.functions import *
exchange_df = df.withColumn("prev_student",lag("student").over(Window.orderBy('id')))
exchange_df = exchange_df.withColumn("next_student",lead("student").over(Window.orderBy('id')))

exchange_df.show()

+---+-------+------------+------------+
| id|student|prev_student|next_student|
+---+-------+------------+------------+
|  1|  Alice|        null|         Bob|
|  2|    Bob|       Alice|     Charlie|
|  3|Charlie|         Bob|       David|
|  4|  David|     Charlie|         Eve|
|  5|    Eve|       David|        null|
+---+-------+------------+------------+



Exchange code based on even odd logic

In [6]:
exchange_df = exchange_df.withColumn('exchange_seating',
                                     when(exchange_df['id']%2 == 1,coalesce(exchange_df['next_student'],exchange_df['student']))
                                     .when(exchange_df['id']%2 == 0,coalesce(exchange_df['prev_student'],exchange_df['student']))
                                     .otherwise(exchange_df['student']))


In [7]:
exchange_df.show()

+---+-------+------------+------------+----------------+
| id|student|prev_student|next_student|exchange_seating|
+---+-------+------------+------------+----------------+
|  1|  Alice|        null|         Bob|             Bob|
|  2|    Bob|       Alice|     Charlie|           Alice|
|  3|Charlie|         Bob|       David|           David|
|  4|  David|     Charlie|         Eve|         Charlie|
|  5|    Eve|       David|        null|             Eve|
+---+-------+------------+------------+----------------+



In [8]:
exchange_df = exchange_df.withColumnRenamed("student","original_seating").drop('prev_student','next_student')

exchange_df.show()

+---+----------------+----------------+
| id|original_seating|exchange_seating|
+---+----------------+----------------+
|  1|           Alice|             Bob|
|  2|             Bob|           Alice|
|  3|         Charlie|           David|
|  4|           David|         Charlie|
|  5|             Eve|             Eve|
+---+----------------+----------------+



code in spark sql

In [9]:
df.createOrReplaceTempView("students")



In [11]:
query = """ SELECT 
    id, 
    student AS Original_seating,
    CASE
        WHEN id % 2 = 1 THEN COALESCE(LEAD(student) OVER (ORDER BY id), student)
        WHEN id % 2 = 0 THEN COALESCE(LAG(student) OVER (ORDER BY id), student)
        ELSE student
    END AS Exchanged_seating
FROM students
"""

In [12]:
# Execute the query and get the result
result = spark.sql(query)

# Show the result
result.show()

+---+----------------+-----------------+
| id|Original_seating|Exchanged_seating|
+---+----------------+-----------------+
|  1|           Alice|              Bob|
|  2|             Bob|            Alice|
|  3|         Charlie|            David|
|  4|           David|          Charlie|
|  5|             Eve|              Eve|
+---+----------------+-----------------+

